# Gestión de Invitados

Este notebook permite administrar la lista de invitados, confirmaciones y análisis de asistencia.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Cargar Datos de Invitados

In [ ]:
df_invitados = pd.read_csv('../datos/invitados.csv')
df_presupuesto = pd.read_csv('../datos/presupuesto.csv')

print(f"Total de grupos: {len(df_invitados)}")
print(f"Total de personas: {df_invitados['personas'].sum()}")
print(f"Personas que requieren hospedaje: {df_invitados['hospedaje'].sum()}")

## 2. Lista Completa de Invitados

In [ ]:
print("\n" + "="*80)
print("                    LISTA COMPLETA DE INVITADOS")
print("="*80 + "\n")

for grupo in ['PUENTE', 'BOGOTA', 'CASA']:
    invitados_grupo = df_invitados[df_invitados['grupo'] == grupo]
    total_personas = invitados_grupo['personas'].sum()
    total_hospedaje = invitados_grupo['hospedaje'].sum()
    
    print(f"\n📍 {grupo} ({len(invitados_grupo)} grupos, {total_personas} personas, {total_hospedaje} hospedaje)")
    print("-" * 80)
    
    for _, row in invitados_grupo.iterrows():
        confirmado = '✅' if row['confirmado'] else '⏳'
        hospedaje_txt = f", {row['hospedaje']}h" if row['hospedaje'] > 0 else ""
        notas_txt = f" - {row['notas']}" if row['notas'] else ""
        print(f"  {confirmado} {row['nombre']:<25} {row['personas']}p{hospedaje_txt}{notas_txt}")

print("\n" + "="*80)

## 3. Estadísticas de Asistencia

In [ ]:
# Calcular estadísticas
total_grupos = len(df_invitados)
total_personas = df_invitados['personas'].sum()
total_hospedaje = df_invitados['hospedaje'].sum()

confirmados = df_invitados[df_invitados['confirmado'] == True]
grupos_confirmados = len(confirmados)
personas_confirmadas = confirmados['personas'].sum()
hospedaje_confirmado = confirmados['hospedaje'].sum()

pendientes = df_invitados[df_invitados['confirmado'] == False]
grupos_pendientes = len(pendientes)
personas_pendientes = pendientes['personas'].sum()
hospedaje_pendiente = pendientes['hospedaje'].sum()

print("\n" + "="*60)
print("           ESTADÍSTICAS DE ASISTENCIA")
print("="*60 + "\n")

print(f"📊 TOTALES:")
print(f"   Grupos invitados:           {total_grupos}")
print(f"   Personas invitadas:         {total_personas}")
print(f"   Requieren hospedaje:        {total_hospedaje}")

print(f"\n✅ CONFIRMADOS:")
print(f"   Grupos confirmados:         {grupos_confirmados} ({grupos_confirmados/total_grupos*100:.1f}%)")
print(f"   Personas confirmadas:       {personas_confirmadas} ({personas_confirmadas/total_personas*100:.1f}%)")
print(f"   Hospedaje confirmado:       {hospedaje_confirmado}")

print(f"\n⏳ PENDIENTES:")
print(f"   Grupos pendientes:          {grupos_pendientes} ({grupos_pendientes/total_grupos*100:.1f}%)")
print(f"   Personas pendientes:        {personas_pendientes} ({personas_pendientes/total_personas*100:.1f}%)")
print(f"   Hospedaje pendiente:        {hospedaje_pendiente}")

print("\n" + "="*60)

## 4. Visualizaciones

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Invitados por grupo
personas_por_grupo = df_invitados.groupby('grupo')['personas'].sum()
colors_grupos = ['#FF6B6B', '#4ECDC4', '#95E1D3']
axes[0, 0].bar(personas_por_grupo.index, personas_por_grupo.values, color=colors_grupos)
axes[0, 0].set_ylabel('Número de Personas', fontsize=12)
axes[0, 0].set_title('Invitados por Grupo de Origen', fontsize=14, fontweight='bold')
for i, v in enumerate(personas_por_grupo.values):
    axes[0, 0].text(i, v, str(v), ha='center', va='bottom', fontsize=12, fontweight='bold')

# 2. Estado de confirmaciones
confirmacion_data = [personas_confirmadas, personas_pendientes]
confirmacion_labels = [f'Confirmados\n({personas_confirmadas})', f'Pendientes\n({personas_pendientes})']
colors_confirmacion = ['#2ecc71', '#f39c12']
axes[0, 1].pie(confirmacion_data, labels=confirmacion_labels, autopct='%1.1f%%',
               startangle=90, colors=colors_confirmacion)
axes[0, 1].set_title('Estado de Confirmaciones', fontsize=14, fontweight='bold')

# 3. Hospedaje por grupo
hospedaje_por_grupo = df_invitados.groupby('grupo')['hospedaje'].sum()
axes[1, 0].bar(hospedaje_por_grupo.index, hospedaje_por_grupo.values, color=colors_grupos)
axes[1, 0].set_ylabel('Personas con Hospedaje', fontsize=12)
axes[1, 0].set_title('Hospedaje Requerido por Grupo', fontsize=14, fontweight='bold')
for i, v in enumerate(hospedaje_por_grupo.values):
    axes[1, 0].text(i, v, str(v), ha='center', va='bottom', fontsize=12, fontweight='bold')

# 4. Comparación personas vs hospedaje por grupo
grupos = df_invitados.groupby('grupo').agg({'personas': 'sum', 'hospedaje': 'sum'})
x = range(len(grupos))
width = 0.35
axes[1, 1].bar([i - width/2 for i in x], grupos['personas'], width, label='Total Personas', color='#3498db')
axes[1, 1].bar([i + width/2 for i in x], grupos['hospedaje'], width, label='Con Hospedaje', color='#e74c3c')
axes[1, 1].set_ylabel('Número de Personas', fontsize=12)
axes[1, 1].set_title('Personas vs Hospedaje por Grupo', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(grupos.index)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 5. Invitados Pendientes por Confirmar

In [ ]:
if len(pendientes) > 0:
    print("\n⏳ INVITADOS PENDIENTES POR CONFIRMAR\n")
    print("="*70)
    
    for grupo in ['PUENTE', 'BOGOTA', 'CASA']:
        pendientes_grupo = pendientes[pendientes['grupo'] == grupo]
        if len(pendientes_grupo) > 0:
            print(f"\n📍 {grupo}:")
            for _, row in pendientes_grupo.iterrows():
                hospedaje_txt = f" ({row['hospedaje']} hospedaje)" if row['hospedaje'] > 0 else ""
                print(f"   - {row['nombre']}: {row['personas']} personas{hospedaje_txt}")
    
    print("\n" + "="*70)
    print(f"\n💡 RECORDATORIO: Contactar a {grupos_pendientes} grupos ({personas_pendientes} personas)")
else:
    print("\n✅ ¡Excelente! Todos los invitados han confirmado su asistencia.")

## 6. Calcular Costos por Persona

In [ ]:
# Calcular costo de comida por persona (del presupuesto)
item_comida = df_presupuesto[df_presupuesto['item'] == 'Comida']
if len(item_comida) > 0 and item_comida.iloc[0]['valor_total'] > 0:
    costo_comida_total = item_comida.iloc[0]['valor_total']
    costo_por_persona = costo_comida_total / total_personas
    
    print("\n💰 COSTOS POR PERSONA\n")
    print("="*60)
    print(f"\nComida:")
    print(f"   Costo total:            ${costo_comida_total:,.0f}")
    print(f"   Personas:               {total_personas}")
    print(f"   Costo por persona:      ${costo_por_persona:,.0f}")
    
    # Proyección por grupo
    print(f"\nProyección por grupo:")
    for grupo in ['PUENTE', 'BOGOTA', 'CASA']:
        personas_grupo = df_invitados[df_invitados['grupo'] == grupo]['personas'].sum()
        costo_grupo = personas_grupo * costo_por_persona
        print(f"   {grupo:<10} ({personas_grupo:>2} personas): ${costo_grupo:>12,.0f}")
else:
    print("\n⚠️  Aún no hay valor registrado para 'Comida' en el presupuesto.")
    print("    Actualiza el archivo datos/presupuesto.csv para calcular costos.")

## 7. Filtros Personalizados

In [ ]:
# Puedes modificar estos filtros según necesites

# Ejemplo 1: Invitados de Bogotá que requieren hospedaje
print("\n🔍 INVITADOS DE BOGOTÁ QUE REQUIEREN HOSPEDAJE\n")
bogota_hospedaje = df_invitados[(df_invitados['grupo'] == 'BOGOTA') & (df_invitados['hospedaje'] > 0)]
for _, row in bogota_hospedaje.iterrows():
    confirmado = '✅' if row['confirmado'] else '⏳'
    print(f"{confirmado} {row['nombre']}: {row['hospedaje']} personas")

# Ejemplo 2: Grupos de más de 2 personas
print("\n🔍 GRUPOS DE MÁS DE 2 PERSONAS\n")
grupos_grandes = df_invitados[df_invitados['personas'] > 2]
for _, row in grupos_grandes.iterrows():
    print(f"{row['nombre']} ({row['grupo']}): {row['personas']} personas")

# Ejemplo 3: Invitados confirmados sin hospedaje
print("\n🔍 INVITADOS CONFIRMADOS SIN HOSPEDAJE\n")
confirmados_sin_hospedaje = df_invitados[(df_invitados['confirmado'] == True) & (df_invitados['hospedaje'] == 0)]
total_sin_hospedaje = confirmados_sin_hospedaje['personas'].sum()
print(f"Total: {len(confirmados_sin_hospedaje)} grupos ({total_sin_hospedaje} personas)")
for _, row in confirmados_sin_hospedaje.iterrows():
    print(f"  - {row['nombre']} ({row['grupo']}): {row['personas']} personas")

## 8. Exportar Lista de Invitados

In [ ]:
from datetime import datetime

# Crear reporte de invitados
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'../datos/reporte_invitados_{timestamp}.xlsx'

with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # Hoja 1: Lista completa
    df_invitados.to_excel(writer, sheet_name='Lista_Completa', index=False)
    
    # Hoja 2: Pendientes
    if len(pendientes) > 0:
        pendientes.to_excel(writer, sheet_name='Pendientes', index=False)
    
    # Hoja 3: Confirmados
    if len(confirmados) > 0:
        confirmados.to_excel(writer, sheet_name='Confirmados', index=False)
    
    # Hoja 4: Resumen por grupo
    resumen_grupos = df_invitados.groupby('grupo').agg({
        'nombre': 'count',
        'personas': 'sum',
        'hospedaje': 'sum'
    }).rename(columns={'nombre': 'num_grupos'})
    resumen_grupos.to_excel(writer, sheet_name='Resumen_Grupos')

print(f"\n✅ Lista de invitados exportada: {filename}")